In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

In [3]:
from june import World 
from june.demography.geography import Geography
from june.demography import Demography
from june.interaction import ContactAveraging
from june.infection import Infection
from june.infection.transmission import TransmissionConstant
from june.groups import Hospitals, Schools, Companies, Households, CareHomes, Cemeteries, Universities
from june.groups.leisure import leisure, Cinemas, Pubs, Groceries
from june.simulator import Simulator
from june.seed import Seed
from june.policy import Policy, Policies
from june import paths
from june.logger.read_logger import ReadLogger
from june.infection.infection import InfectionSelector
from june.world import generate_world_from_hdf5, generate_world_from_geography

No --data argument given - defaulting to:
/home/florpi/JUNE/data
No --configs argument given - defaulting to:
/home/florpi/JUNE/configs


# Initialize world

To initialize a certain world, we need to add the different components we want to have in it. First we specify what super areas (msoa) we want to create. We have included these ones, because they are known to contain hospitals, schools, care homes, and companies.

After creating the geography, we create the different components the worlds need to have such as care homes, companies ...

In [4]:
%%time 

geography = Geography.from_file(
        {
            "super_area": [
                "E02003282",
                "E02001720",
                "E00088544",
                "E02002560",
                "E02002559",
                "E02004314",
            ]
        }
)

geography.hospitals = Hospitals.for_geography(geography)
geography.schools = Schools.for_geography(geography)
geography.companies = Companies.for_geography(geography)
geography.care_homes = CareHomes.for_geography(geography)
geography.universities = Universities.for_super_areas(geography.super_areas)
world = generate_world_from_geography(geography, include_households=True, include_commute=True)


2020-06-23 17:15:57,898 - june.demography.geography - INFO - There are 129 areas and 5 super_areas in the world.
2020-06-23 17:15:57,909 - june.groups.hospital - INFO - There are 1 hospitals in this geography.
2020-06-23 17:15:57,967 - june.groups.school - INFO - There are 19 schools in this geography.
2020-06-23 17:15:58,003 - june.groups.school - INFO - No school for the age 0 in this world.
2020-06-23 17:15:58,009 - june.groups.school - INFO - No school for the age 1 in this world.
2020-06-23 17:15:58,023 - june.groups.school - INFO - No school for the age 2 in this world.
2020-06-23 17:15:58,364 - june.groups.carehome - INFO - There are 9 care_homes in this geography.
2020-06-23 17:16:05,777 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.
2020-06-23 17:16:08,635 - june.distributors.worker_distributor - INFO - There are 0 who had to be told to stay real
CPU times: user 34.2 s, sys: 1.2 s, total: 35.4 s
Wall time: 36 s


In [5]:
len(world.people)

46784

### If it took a long time to run the previous command, it might be a good idea to save the world to reuse it later.

In [6]:
#world.to_hdf5("world.hdf5")

If we would like to load the world we saved, we just do

In [7]:
#world = generate_world_from_hdf5("world.hdf5")

you have now a beautiful pre-pandemic world. 

## Commute, travel and leisure

We can add cinemas, pubs, groceries, etc. as well as commute in major cities with

In [8]:
# leisure
world.cinemas = Cinemas.for_geography(geography)
world.pubs = Pubs.for_geography(geography)
world.groceries = Groceries.for_super_areas(world.super_areas,
                                            venues_per_capita=1/500)

In [9]:
# commute
#world.initialise_commuting()

We are also going to need some cemeteries...geography.cemeteries = Cemeteries()


In [10]:
world.cemeteries = Cemeteries()

# Adding the infection

Now, you can play around with different models of infections. The building blocks for an infection are: Transmission and Symptoms. For now, only SymptomsConstant and TransmissionConstant are in a state to be used.

In [11]:
selector = InfectionSelector.from_file()

In [12]:
selector

# Adding the interaction

In [13]:
interaction = ContactAveraging.from_file(selector=selector)
#interaction = DefaultInteraction.from_file(selector=selector)

Beta are the intensities of the interaction taking place at the different groups

In [14]:
interaction.beta

{'box': 1,
 'pub': 1,
 'grocery': 1,
 'cinema': 1,
 'commute_unit': 1.0,
 'commute_city_unit': 1.0,
 'hospital': 1.0,
 'care_home': 1.0,
 'company': 1.0,
 'school': 1.0,
 'household': 1.0,
 'university': 1.0}

to modify these, simply do

In [15]:
interaction.beta['household'] *= 2

In [16]:
interaction.beta

{'box': 1,
 'pub': 1,
 'grocery': 1,
 'cinema': 1,
 'commute_unit': 1.0,
 'commute_city_unit': 1.0,
 'hospital': 1.0,
 'care_home': 1.0,
 'company': 1.0,
 'school': 1.0,
 'household': 2.0,
 'university': 1.0}

moreover this interaction module uses contact matrices, that are different for different groups. These contact matrices shouldnt be modified for now. However they are a combination of conversational contact matrices, and physical contact matrices (see the BBC pandemic paper, from where these matrices are extracted https://www.medrxiv.org/content/10.1101/2020.02.16.20023754v2)

There is a parameter, ``alpha`` ($\alpha$), that combines these two matrices in the following way,


$\beta M \left(1 + (\alpha -1) \right) P$

where $\beta$ is the intensity of the interaction, and $P$ the physical contact matrix. A larger $\alpha$ produces more physical contacts. It is an overall number, non dependent of the particular group.


In [17]:
interaction.alpha_physical

2.0

In [18]:
interaction.alpha_physical /= 2

In [19]:
interaction.alpha_physical

1.0

# Seed the disease

There are two options implemented in the seed at the moment, either you specify the number of cases and these are then homogeneously distributed by population to the different areas, or you use UK data on cases per region. For now use the first case.

In [20]:
seed = Seed(world.super_areas, selector,)

In [21]:
n_cases = 50
seed.unleash_virus(n_cases) # play around with the initial number of cases

# Set policies

In [22]:

social_distance = Policy(policy="social_distance",
                         start_time=datetime(2020, 3, 25), 
                         end_time=datetime(2020, 4, 1))
policies = Policies.from_file([social_distance])


# Run the simulation

Since the timer configuration is a bit cumbersome, it is read from the config file at ``configs/config_example.yaml``

In [23]:
CONFIG_PATH = "../configs/config_example.yaml"

leisure_instance = leisure.generate_leisure_for_config(
world=world, config_filename = CONFIG_PATH
)
simulator = Simulator.from_file(
     world, interaction, selector, 
    config_filename = CONFIG_PATH,
    leisure = leisure_instance,
    policies = policies
)

In [ ]:
%%time
simulator.run()

2020-06-23 17:17:29,936 - june.simulator - INFO - Starting group_dynamics for 70 days at day 0
2020-06-23 17:17:29,937 - june.simulator - INFO - starting the loop ..., at 0 days, to run for 70 days
2020-06-23 17:17:34,295 - june.simulator - INFO - Date = 2020-03-07 04:00:00, number of deaths =  0, number of infected = 50
2020-06-23 17:17:38,788 - june.simulator - INFO - Date = 2020-03-07 08:00:00, number of deaths =  0, number of infected = 50
2020-06-23 17:17:39,438 - june.simulator - INFO - Date = 2020-03-07 12:00:00, number of deaths =  0, number of infected = 50
2020-06-23 17:17:43,649 - june.simulator - INFO - Date = 2020-03-08 00:00:00, number of deaths =  0, number of infected = 50
2020-06-23 17:17:46,690 - june.simulator - INFO - Date = 2020-03-08 04:00:00, number of deaths =  0, number of infected = 50
2020-06-23 17:17:49,553 - june.simulator - INFO - Date = 2020-03-08 08:00:00, number of deaths =  0, number of infected = 50
2020-06-23 17:17:50,013 - june.simulator - INFO - Da

2020-06-23 17:19:18,505 - june.simulator - INFO - Date = 2020-03-19 01:00:00, number of deaths =  0, number of infected = 9025
2020-06-23 17:19:25,261 - june.simulator - INFO - Date = 2020-03-19 05:00:00, number of deaths =  0, number of infected = 10702
2020-06-23 17:19:33,052 - june.simulator - INFO - Date = 2020-03-19 09:00:00, number of deaths =  0, number of infected = 12454
2020-06-23 17:19:35,872 - june.simulator - INFO - Date = 2020-03-19 10:00:00, number of deaths =  0, number of infected = 12647
2020-06-23 17:19:36,752 - june.simulator - INFO - Date = 2020-03-19 13:00:00, number of deaths =  0, number of infected = 13591
2020-06-23 17:19:37,876 - june.simulator - INFO - Date = 2020-03-20 00:00:00, number of deaths =  0, number of infected = 14756
2020-06-23 17:19:39,063 - june.simulator - INFO - Date = 2020-03-20 01:00:00, number of deaths =  0, number of infected = 14927
2020-06-23 17:19:49,891 - june.simulator - INFO - Date = 2020-03-20 05:00:00, number of deaths =  0, numb

2020-06-23 17:21:37,753 - june.simulator - INFO - Date = 2020-03-31 01:00:00, number of deaths =  16, number of infected = 46252
2020-06-23 17:21:38,870 - june.simulator - INFO - Date = 2020-03-31 05:00:00, number of deaths =  20, number of infected = 46250
2020-06-23 17:21:39,601 - june.simulator - INFO - Date = 2020-03-31 09:00:00, number of deaths =  22, number of infected = 46252
2020-06-23 17:21:41,233 - june.simulator - INFO - Date = 2020-03-31 10:00:00, number of deaths =  22, number of infected = 46245
2020-06-23 17:21:41,923 - june.simulator - INFO - Date = 2020-03-31 13:00:00, number of deaths =  22, number of infected = 46227
2020-06-23 17:21:42,608 - june.simulator - INFO - Date = 2020-04-01 00:00:00, number of deaths =  23, number of infected = 46118
2020-06-23 17:21:43,553 - june.simulator - INFO - Date = 2020-04-01 01:00:00, number of deaths =  23, number of infected = 46105
2020-06-23 17:21:44,469 - june.simulator - INFO - Date = 2020-04-01 05:00:00, number of deaths = 

While the simulation runs (and afterwards) we can launch the visualization webpage by running
```python june/visualizer.py path/to/results``` 

# Getting the results

All results are stored in a json file specified in the ``save_path`` variable in the config file. We can also access it from ``world.logger`` directly.

In [ ]:
import pandas as pd

In [ ]:
read = ReadLogger()

## Hospital data and how it changed over time

In [ ]:
hospitals_df = read.load_hospital_capacity()

In [ ]:
hospitals_df.head(3)

In [ ]:
hospitals_characteristics_df = read.load_hospital_characteristics()

In [ ]:
hospitals_characteristics_df

## where did infections happen?

In [ ]:
loc_df = read.get_locations_infections()

In [ ]:
locations_per_day = read.locations_df.groupby(pd.Grouper(freq='D')).sum()

In [ ]:
all_infection_places = set(locations_per_day.location.sum())

In [ ]:
def n_infections(row, infection_place):
    return sum([row.counts[i] for i, x in enumerate(row.location) if x == infection_place])


In [ ]:
for infection_place in all_infection_places:
    locations_per_day[str(infection_place)] = locations_per_day.apply(
        lambda x: n_infections(x, infection_place),
        axis=1
    )

In [ ]:
locations_per_day = locations_per_day.drop(columns=['location',
                                                    'counts'])

In [ ]:
locations_per_day = locations_per_day.div(
    locations_per_day.sum(axis=1), axis=0
)


In [ ]:
locations_per_day.plot.area( alpha=0.5)
plt.legend(bbox_to_anchor=(1,1))
plt.ylabel('Percent of infections')

In [ ]:
import matplotlib.ticker as mtick

ax = loc_df['percentage_infections'].sort_values().plot.bar()
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.ylabel('Percentage of infections at location')
plt.xlabel('location')
#plt.yscale('log')

## rate of infection

In [ ]:
r_df = read.get_r()

In [ ]:
r_df.plot()
plt.axvspan(policies.social_distancing_start, 
            policies.social_distancing_end,
            facecolor='y', alpha=0.2)
plt.axhline(y=1, linestyle='dashed', color='gray')
plt.xlabel('Date')
plt.ylabel('R')

## World infection curves, and by super area

In [ ]:
world_df = read.world_summary()

In [ ]:
world_df['infected'].plot()
plt.axvspan(policies.social_distancing_start, 
            policies.social_distancing_end,
            facecolor='y', alpha=0.2)
plt.ylabel('# Infected')

## Seroprevalence

In [ ]:
(100*world_df['new_infections'].cumsum()/read.n_people).plot()
plt.axvspan(policies.social_distancing_start, 
            policies.social_distancing_end,
            facecolor='y', alpha=0.2)
plt.axhline(y=2, linestyle='dashed', color='indianred')
plt.ylabel('Seroprevalence (% infected)')

In [ ]:
world_df.plot()
plt.axvspan(policies.social_distancing_start, 
            policies.social_distancing_end,
            facecolor='y', alpha=0.2)
plt.legend(bbox_to_anchor=(1,1))


In [ ]:
area_df = read.super_area_summary()

In [ ]:
area_df[area_df.super_area == "E02003282"].plot()
plt.legend(bbox_to_anchor=(1,1))



## World infection curves per age group

In [ ]:
ages_df = read.age_summary([0,10,20,30,40,
                  50,60,70,80,90,100])

In [ ]:
for name, group in ages_df.groupby('age_range'):
    group['infected'].plot(label=name)
plt.axvspan(policies.social_distancing_start, 
            policies.social_distancing_end,
            facecolor='y', alpha=0.2)
plt.legend(bbox_to_anchor=(1,1))


In [ ]:
for name, group in ages_df.groupby('age_range'):
    group['dead'].cumsum().plot(label=name)
plt.legend()

In [ ]:
for name, group in ages_df.groupby('age_range'):
    group['susceptible'].plot(label=name)
plt.legend()

In [ ]:
for name, group in ages_df.groupby('age_range'):
    group['hospital_admissions'].cumsum().plot(label=name)
plt.legend()

In [ ]:
for name, group in ages_df.groupby('age_range'):
    group['intensive_care_admissions'].cumsum().plot(label=name)
plt.legend()

## Draw some of the symptoms trajectories

In [ ]:
random_trajectories = read.draw_symptom_trajectories(window_length=600,
                                        n_people=10)

In [ ]:
from june.infection import SymptomTag

In [ ]:
symptoms_values = [tag.value for tag in SymptomTag]
symptoms_names = [tag.name for tag in SymptomTag]


In [ ]:
for df_person in random_trajectories:
    df_person['symptoms'].plot()
plt.ylabel('Symptoms Trajectory')
_ = plt.yticks(symptoms_values, symptoms_names)
plt.xlabel('Date')

In [ ]:
for df_person in random_trajectories:
    df_person['n_secondary_infections'].plot()
plt.ylabel('Number of secondary infections')

plt.xlabel('Date')